In [2]:
!pip install polars

     |████████████████████████████████| 32.5 MB 111 kB/s  eta 0:00:01


In [14]:
import polars as p1

# question 1
lazy_df = p1.scan_csv("data/202306-divvy-tripdata.csv", dtypes={"start_station_id": p1.Utf8, "end_station_id": p1.Utf8}, try_parse_dates=True)


/tmp/ipykernel_10345/1967509838.py:4: DeprecationWarning: The argument `dtypes` for `scan_csv` is deprecated. It has been renamed to `schema_overrides`.
  lazy_df = p1.scan_csv("data/202306-divvy-tripdata.csv", dtypes={"start_station_id": p1.Utf8, "end_station_id": p1.Utf8}, try_parse_dates=True)


In [ ]:
# question 2

# part 1
lazy_df = lazy_df.with_columns([
    p1.col("ride_id").cast(p1.Utf8),
    p1.col("rideable_type").cast(p1.Utf8),
    p1.col("member_casual").cast(p1.Utf8),
    
    p1.col("started_at"),
    p1.col("ended_at"),
      
    p1.col("start_station_name").cast(p1.Utf8),
    p1.col("start_station_id").cast(p1.Utf8),
    p1.col("end_station_name").cast(p1.Utf8),
    p1.col("end_station_id").cast(p1.Utf8),

    p1.col("start_lat").cast(p1.Float64),
    p1.col("start_lng").cast(p1.Float64),
    p1.col("end_lat").cast(p1.Float64),
    p1.col("end_lng").cast(p1.Float64)
])

df = lazy_df.collect()

print(df.schema)

Schema([('ride_id', String), ('rideable_type', String), ('started_at', Datetime(time_unit='us', time_zone=None)), ('ended_at', Datetime(time_unit='us', time_zone=None)), ('start_station_name', String), ('start_station_id', String), ('end_station_name', String), ('end_station_id', String), ('start_lat', Float64), ('start_lng', Float64), ('end_lat', Float64), ('end_lng', Float64), ('member_casual', String)])


In [24]:
# Question 2nd part2

rides_per_day = (
    lazy_df.with_columns([
    p1.col("started_at").dt.date().alias("date")
])
.group_by("date")
.agg(p1.count().alias("ride_count"))
.sort("ride_count", descending=True)
.collect()
)
print(rides_per_day)

shape: (30, 2)
┌────────────┬────────────┐
│ date       ┆ ride_count │
│ ---        ┆ ---        │
│ date       ┆ u32        │
╞════════════╪════════════╡
│ 2023-06-19 ┆ 58         │
│ 2023-06-22 ┆ 53         │
│ 2023-06-30 ┆ 51         │
│ 2023-06-10 ┆ 50         │
│ 2023-06-23 ┆ 45         │
│ …          ┆ …          │
│ 2023-06-26 ┆ 21         │
│ 2023-06-27 ┆ 20         │
│ 2023-06-08 ┆ 16         │
│ 2023-06-14 ┆ 16         │
│ 2023-06-11 ┆ 13         │
└────────────┴────────────┘


/tmp/ipykernel_10345/3128129741.py:8: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  .agg(p1.count().alias("ride_count"))


In [ ]:
# Question2 part 3

lazy_df = lazy_df.with_columns([
    p1.col("started_at").dt.year().alias("Year"),
    p1.col("started_at").dt.week().alias("Week_number")
])

rides_per_week = (
    lazy_df.group_by(["Year", "Week_number"])
    .agg(p1.len().alias("Total_rides_per_week"))
)

weekly_stats = (
    rides_per_week
    .select([
        # p1.col("Year"),
        # p1.col("Week_number"),
        p1.col("Total_rides_per_week").mean().alias("Avg_rides_per_week"),
        p1.col("Total_rides_per_week").max().alias("Max_rides_per_week"),
        p1.col("Total_rides_per_week").min().alias("Min_rides_per_week")
    ])
    .collect()
)

print(weekly_stats)


shape: (1, 3)
┌────────────────────┬────────────────────┬────────────────────┐
│ Avg_rides_per_week ┆ Max_rides_per_week ┆ Min_rides_per_week │
│ ---                ┆ ---                ┆ ---                │
│ f64                ┆ u32                ┆ u32                │
╞════════════════════╪════════════════════╪════════════════════╡
│ 199.8              ┆ 286                ┆ 142                │
└────────────────────┴────────────────────┴────────────────────┘


In [ ]:
# Question2 Part4

rides_per_day = (
    lazy_df
    .with_columns(p1.col("started_at").dt.date().alias("date"))
    .group_by("date")
    .agg(p1.len().alias("Total_rides"))
    .sort("date")
    .collect()
)

rides_per_day = rides_per_day.with_columns([
    p1.col("Total_rides").shift(7).alias("Rides_last_week")
])

rides_per_day = rides_per_day.with_columns([
    (p1.col("Total_rides").cast(p1.Int64) - p1.col("Rides_last_week").cast(p1.Int64))
        .alias("Diff_from_last_week")
])

rides_per_day


date,Total_rides,Rides_last_week,Diff_from_last_week
date,u32,u32,i64
2023-06-01,43,null,null
2023-06-02,40,null,null
2023-06-03,33,null,null
2023-06-04,26,null,null
2023-06-05,31,null,null
…,…,…,…
2023-06-26,21,58,-37
2023-06-27,20,28,-8
2023-06-28,35,29,6
